导入相关的包

In [95]:
import numpy as np
from matplotlib import pyplot as plt

定义计算optimal solution的函数:
$$
-\sum_{i = 1}^{n} \log (\alpha_i + x_i)
$$

In [96]:
def calculate_target(alpha,x):
    return -(np.log2(alpha+x).sum())

定义fill_water函数

同之前一样首先初始化 $\nu$ 的值为 2.0 (该$\nu$值是我们经过几次实验之后的经验值), $error$ 的值为1.0, 从0开始迭代，并开创 $targets[], errors[]$ 来记录每一次迭代之后的solution以及相应的errors的变化

根据牛顿法的思路，我们选择$\Delta x_{nt} = -\nabla^2 f(x)^{-1} \nabla f(x)$，因此我们先求出一阶导与二阶导，并将其作为更新指标

此处出现了
* if abs(d2F_dnu2) < epsilon:
* d2F_dnu2 = epsilon

是由于当d2F_dnu2的值非常小时，代码会出现Runtime error的warning，也即分母不能为零，因此设置了一个最小值$epsilon = 10^{-8}$

但非常可惜的是，在Gradient Descent求一阶导时代码已经运行得很慢了，此处还要求二阶导，其运行时间相当的长，因此也未在该.ipynb中给出各代码块的输出

为了方便不同时候的需要，我们加入一个判断语句，通过可选的bool参数track, 来决定我们最后的return:
* 如果我们的输出不存在问题，可将track选为false, 以此来简化输出，仅仅输出我们最后的optimmal $x^*$
* 如果我们最后的输出不知为何出了问题，或者说我们想要看到target和error在每一步iterations之后的变化，我们可将track选为true,这样就不仅仅会输出 optimal $x^*$,还会输出数组$target[] 和 error[]$



In [97]:
def fill_water(alpha, total_water, precision, track=False):
    nu = 2.0
    error = 1.0
    iteration = 0
    targets = []
    errors = []
    epsilon = 1e-8
    
    while error > precision:
        x = np.maximum(0, 1/nu - alpha)
        total_power = x.sum()
        error = np.abs(total_power - total_water)
        
        if track:
            targets.append(calculate_target(alpha, x))
            errors.append(error)
        
        dF_dnu = np.sum(1 / (nu**2))  # First derivative
        d2F_dnu2 = np.sum(-2  / (nu**3))  # Second derivative

        if abs(d2F_dnu2) < epsilon:
            d2F_dnu2 = epsilon
        
        nu = nu - dF_dnu / d2F_dnu2
        
        iteration += 1
        
    if track:
        return x, targets, errors
    return x

此处我们先规定随机生成的 $\alpha$ 的区间，并规定 $total_water = 1.0$ 和 $dimension = 10$，换到实际问题中便可以是我们规定总水量为1吨，总共有10个channels，
再给出我们像要达到的精确度，此处设为了$10^{-6}$

然后我们再使用random函数随机生成了10个$\alpha_i$值，并将它们打印出来

In [ ]:
alpha_range = [0.0,1.0]
total_water = 1.0
dimension = 10

precision = 1e-6

alpha = np.random.uniform(low = alpha_range[0],high = alpha_range[1],size = (dimension,1))
print(alpha)

然后我们利用定义的$fill\_water$函数求出 optimal $x^*$并将其打印出来，以及将 $\sum_{i=1}^{n} x_i^*$打印出来（可以此比较其与1的差值以见是否达到要求）

同时我们求出水位 $horizontal\_ line$ 其实也即是 $1/\nu^*$,并将其打印出来

In [ ]:
x = fill_water(alpha = alpha, total_water=total_water,precision=precision)
print(x)
print(x.sum())

horizontal_line = (alpha+x).min()
print(horizontal_line)

此处定义 $visualize\_ water$函数以对该问题的最终结果进行可视化呈现:

The height of each patch is given by $\alpha_i$. The region is flooded to a level $1/\nu^*$ which uses a total quantity of water equal to one. The height of the water (shown blue) above each patch is the optimal value of $x^*$.

In [88]:
def visualize_water(alpha,x,horizontal_line):
    alpha = alpha.squeeze()
    x = x.squeeze()

    x_range = range(1,x.shape[0]+1)
    plt.xticks(x_range)
    plt.bar(x_range,alpha,color = '#ff9966',width = 1.0,edgecolor = '#ff9966')
    plt.bar(x_range,x,bottom = alpha,color = '#4db8ff',width = 1.0)

    plt.axhline(y = horizontal_line,linewidth =1,color = 'k')
    plt.show()

In [ ]:
visualize_water(alpha,x,horizontal_line)

此处与$fill\_water$函数中的关于 $track$ 的if 判断语句相对应，可以将最终的 $x^*$ 以及每一次iteration之后的targets和errors输出，便于在代码出现问题之后debug

同时，此处可查看迭代的次数

In [ ]:
x,targets,errors = fill_water(alpha = alpha,total_water=total_water,precision=precision,track = True)
print(x)
print(targets)
print(errors)

print('iteration:',len(errors))

此处定义 $visualize\_targets\_and\_errors$函数，对在迭代过程中的targets和errors的变化进行可视化呈现，以观察实验效果

In [91]:
def visualize_targets_and_errors(targets,errors):
    x = range(len(targets))
    plt.plot(x,targets,label = 'targets')
    plt.plot(x,errors,label = 'errors')

    plt.legend(loc = 'best')

    plt.show()

In [ ]:
visualize_targets_and_errors(targets,errors)

⼀顿操作之后，我们很自然地想确认自己的结果是不是正确的，但对于该问题，由于$\alpha$生成的随机性，“标准答案”并不好寻找，我们就可以通过判断我们的实现的算法是否⽐随机⽣成的解要好，以此来判断我们的解的正确性

此处我们定义了一个 $monkey_search$函数以生成随机解，并将其可视化呈现。其中橙线是我们得到的optimal solution of $-\sum_{i = 1}^{n} \log (\alpha_i + x_i)$,蓝点即是每一只“猴子”确定的随机解

In [93]:
def monkey_search(alpha):
    while True:
        monkey_solutions = np.random.dirichlet(np.ones(10),size = 1).reshape(-1,1)
        if np.less(monkey_solutions,0).any():
            continue
        return monkey_solutions
    
def visualize_monkey_search(alpha,monkey_amount,optimal):
    monkey_solutions = [calculate_target(alpha,monkey_search(alpha))\
                        for _ in range(monkey_amount)]
    plt.scatter(range(monkey_amount),monkey_solutions)
    plt.axhline(y = optimal,linewidth = 1,color = 'r')

    plt.show()

In [ ]:
optimal_line= targets[len(targets)-1]
visualize_monkey_search(alpha,1000,optimal_line)